[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github.com/RichmondAlake/agent_memory_course/blob/main/mem0/memory_augmented_agent_with_mem0_mongodb.ipynb)


In [1]:
! pip install -qU mem0ai

In [2]:
# Constants
CURRENT_USER_ID = "user-123"

In [3]:
import getpass
import os

# Function to securely get and set environment variables
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

In [ ]:
set_env_securely("OPENAI_API_KEY", "Enter your OPENAI_API_KEY: ")

In [ ]:
set_env_securely("MONGODB_URI", "Enter your MongoDB URI: ")

In [6]:
config = {
    "vector_store": {
        "provider": "mongodb",
        "config": {
            "db_name": "mem0_agent_memory",
            "collection_name": "extracted_memories",
            "mongo_uri": os.environ["MONGODB_URI"]
        }
    }
}


semantic_memory_config = {
    "vector_store": {
        "provider": "mongodb",
        "config": {
            "db_name": "mem0_agent_memory",
            "collection_name": "semantic_memory",
            "mongo_uri": os.environ["MONGODB_URI"]
        }
    }
}

In [8]:
from mem0 import Memory
memory = Memory.from_config(config)
semantic_memory = Memory.from_config(semantic_memory_config)

INFO:mem0.vector_stores.mongodb:Search index 'extracted_memories_vector_index' already exists in collection 'extracted_memories'.
INFO:mem0.vector_stores.mongodb:Search index 'mem0migrations_vector_index' already exists in collection 'mem0migrations'.
INFO:mem0.vector_stores.mongodb:Retrieved document with ID '2a50db03-70cb-4ce8-85ce-cf68cf30343e'.
INFO:mem0.vector_stores.mongodb:MongoClient connection closed.
INFO:mem0.vector_stores.mongodb:MongoClient connection closed.
INFO:mem0.vector_stores.mongodb:Search index 'semantic_memory_vector_index' already exists in collection 'semantic_memory'.
INFO:mem0.vector_stores.mongodb:Search index 'mem0migrations_vector_index' already exists in collection 'mem0migrations'.
INFO:mem0.vector_stores.mongodb:Retrieved document with ID '2a50db03-70cb-4ce8-85ce-cf68cf30343e'.
INFO:mem0.vector_stores.mongodb:MongoClient connection closed.
INFO:mem0.vector_stores.mongodb:MongoClient connection closed.


In [9]:
from openai import OpenAI
from typing import Optional, Dict

openai_client = OpenAI()

def patched_search_vector_store(self, query, filters, limit, threshold: Optional[float] = None):
    """
    Patched version of _search_vector_store that uses the correct MongoDB parameter name.
    
    This fixes the TypeError: MongoDB.search() got an unexpected keyword argument 'vectors'
    by using 'query_vector' instead of 'vectors' to match the actual MongoDB implementation.
    """
    # Generate embeddings for the query
    embeddings = self.embedding_model.embed(query, "search")
    
    # Call MongoDB search with correct parameter name: query_vector instead of vectors
    memories = self.vector_store.search(query=query, query_vector=embeddings, limit=limit, filters=filters)
    
    # Process the results to match expected format
    promoted_payload_keys = [
        "user_id",
        "agent_id", 
        "run_id",
        "actor_id",
        "role",
    ]

    core_and_promoted_keys = {"data", "hash", "created_at", "updated_at", "id", *promoted_payload_keys}

    # Apply threshold filtering if specified
    if threshold is not None:
        filtered_memories = []
        for memory_item in memories:
            if hasattr(memory_item, 'score') and memory_item.score >= threshold:
                filtered_memories.append(memory_item)
        memories = filtered_memories

    # Serialize memories to expected format
    serialized_memories = []
    for memory_item in memories:
        serialized_memory = {}
        
        # Handle the core fields
        for key in core_and_promoted_keys:
            if key == "data":
                # The actual memory content is in the payload
                serialized_memory[key] = getattr(memory_item, 'payload', {}).get('data', None)
            else:
                serialized_memory[key] = getattr(memory_item, key, None)

        # Add score and memory content
        serialized_memory["score"] = getattr(memory_item, "score", None)
        serialized_memory["memory"] = getattr(memory_item, 'payload', {}).get('data', '')
        serialized_memory["metadata"] = getattr(memory_item, 'payload', {}).get('metadata', {})

        # Add promoted payload keys
        payload = getattr(memory_item, 'payload', {})
        for key in promoted_payload_keys:
            value = payload.get(key, None)
            if value is not None:
                serialized_memory[key] = value

        serialized_memories.append(serialized_memory)

    return serialized_memories

def create_patched_add_to_vector_store(original_method):
    """
    Factory function to create a patched _add_to_vector_store method.
    This ensures each memory instance gets its own reference to the original method.
    """
    def patched_add_to_vector_store(self, messages, metadata, filters, infer):
        """
        Patched version of _add_to_vector_store that fixes the MongoDB search parameter issue.
        
        This method is called during memory.add() operations and also uses the incorrect 'vectors' parameter.
        """
        # Import the logger from the original module
        import logging
        logger = logging.getLogger(__name__)
        
        # Get the ACTUAL MongoDB vector store search method (not our patched version)
        mongodb_vector_store = self.vector_store
        
        # Get the original MongoDB search method directly
        original_mongodb_search = mongodb_vector_store.__class__.search
        
        def patched_vector_store_search(self_vs, query, vectors, limit, filters):
            # Fix the parameter name: vectors -> query_vector
            return original_mongodb_search(self_vs, query=query, query_vector=vectors, limit=limit, filters=filters)
        
        # Temporarily patch the vector store search method
        original_vector_store_search = mongodb_vector_store.search
        mongodb_vector_store.search = patched_vector_store_search.__get__(mongodb_vector_store, mongodb_vector_store.__class__)
        
        try:
            # Call the original _add_to_vector_store method as a bound method
            result = original_method.__get__(self, type(self))(messages, metadata, filters, infer)
            return result
        finally:
            # Restore the original vector store search method
            mongodb_vector_store.search = original_vector_store_search
    
    return patched_add_to_vector_store

In [10]:
# Apply the monkey patches at both class and instance level
from mem0.memory.main import Memory

# Store original methods for each instance BEFORE patching
memory_original_search = memory._search_vector_store
memory_original_add = memory._add_to_vector_store
semantic_memory_original_search = semantic_memory._search_vector_store
semantic_memory_original_add = semantic_memory._add_to_vector_store

# Store original class methods before any patching
original_class_search = Memory._search_vector_store
original_class_add = Memory._add_to_vector_store

# Apply class-level patches (affects new instances)
Memory._search_vector_store = patched_search_vector_store
Memory._add_to_vector_store = create_patched_add_to_vector_store(original_class_add)

# Create instance-specific patched add methods
memory_patched_add = create_patched_add_to_vector_store(memory_original_add)
semantic_memory_patched_add = create_patched_add_to_vector_store(semantic_memory_original_add)

# Apply patches to existing instances
memory._search_vector_store = patched_search_vector_store.__get__(memory, memory.__class__)
memory._add_to_vector_store = memory_patched_add.__get__(memory, memory.__class__)

semantic_memory._search_vector_store = patched_search_vector_store.__get__(semantic_memory, semantic_memory.__class__)
semantic_memory._add_to_vector_store = semantic_memory_patched_add.__get__(semantic_memory, semantic_memory.__class__)

print("🔧 MongoDB search and add methods patched successfully for both memory instances!")

🔧 MongoDB search and add methods patched successfully for both memory instances!


## Data Ingestion

In [11]:
import sys

# Method 1: Add the project root directory to Python path
project_root = "/Users/richmondalake/Desktop/Projects/open_source/agent_memory_course"
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utilities.pdf_chunker import ingest_pdf_and_chunk

url = "https://arxiv.org/pdf/2404.13501"

# Ingest and chunk the PDF
chunks = ingest_pdf_and_chunk(url)

Loading PDF with LangChain...
Chunking text...
Successfully created 197 chunks


In [ ]:
# Ingest chunks into semantic memory one by one to avoid timeout issues
print(f"📄 Ingesting {len(chunks)} chunks into semantic memory...")

successful_ingestions = 0
failed_ingestions = 0

for i, chunk in enumerate(chunks):
    try:
        # Process each chunk individually
        chunk_content = chunk["value"]["content"]
        
        # Skip very short chunks (less than 50 characters)
        if len(chunk_content.strip()) < 50:
            continue
            
        # Create a single message for this chunk
        message = {"role": "user", "content": chunk_content}
        
        # Add to semantic memory
        result = semantic_memory.add([message], user_id=CURRENT_USER_ID, infer=False)
        successful_ingestions += 1
        
        if (i + 1) % 10 == 0:  # Progress update every 10 chunks
            print(f"   Processed {i + 1}/{len(chunks)} chunks...")
            
    except Exception as e:
        print(f"   ❌ Failed to ingest chunk {i}: {str(e)[:100]}...")
        failed_ingestions += 1
        continue

print(f"✅ Ingestion complete!")
print(f"   - Successfully ingested: {successful_ingestions} chunks")
print(f"   - Failed: {failed_ingestions} chunks")
print(f"   - Total processed: {successful_ingestions + failed_ingestions}/{len(chunks)}")

In [13]:

def chat_with_memories(message: str, user_id: str = "default_user") -> str:
    """
    Chat function that retrieves relevant memories and generates responses.
    
    Args:
        message (str): User's input message
        user_id (str): User identifier for memory retrieval
        
    Returns:
        str: Assistant's response based on memories and query
    """
    try:
        # Retrieve relevant memories using the fixed search method
        relevant_memories = memory.search(query=message, user_id=user_id, limit=3)
        relevant_memories_from_semantic_memory = semantic_memory.search(query=message, user_id=user_id, limit=3)
        
        # Format memories for display and context
        memories_str = "\n".join(f"- {entry['memory']}" for entry in relevant_memories["results"])
        memories_str_from_semantic_memory = "\n".join(f"- {entry['memory']}" for entry in relevant_memories_from_semantic_memory["results"])
        print("Memories:")
        print(memories_str if memories_str else "No relevant memories found.")
        print("Semantic Memories:")
        print(memories_str_from_semantic_memory if memories_str_from_semantic_memory else "No relevant semantic memories found.")

        # Generate Assistant response with memory context
        system_prompt = f"You are a helpful AI. Answer the question based on the query and memories.\nUser Memories:\n{memories_str}\nSemantic Memories:\n{memories_str_from_semantic_memory}"
        messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": message}]
        
        response = openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages)
        assistant_response = response.choices[0].message.content

        # Create new memories from the conversation
        messages.append({"role": "assistant", "content": assistant_response})
        memory.add(messages, user_id=user_id)

        return assistant_response
        
    except Exception as e:
        print(f"Error in chat_with_memories: {e}")
        # Fallback response without memory
        messages = [{"role": "user", "content": message}]
        response = openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages)
        return response.choices[0].message.content

def main():
    """Interactive chat loop"""
    print("Chat with AI (type 'exit' to quit)")
    print("This AI has memory and can remember your conversation!")
    
    while True:
        user_input = input("\nYou: ").strip()
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break
        
        try:
            response = chat_with_memories(user_input)
            print(f"AI: {response}")
        except Exception as e:
            print(f"Error: {e}")

if __name__ == "__main__":
    main()

Chat with AI (type 'exit' to quit)
This AI has memory and can remember your conversation!


INFO:mem0.vector_stores.mongodb:Retrieved document with ID '2a50db03-70cb-4ce8-85ce-cf68cf30343e'.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 0 documents.
INFO:mem0.vector_stores.mongodb:Retrieved document with ID '2a50db03-70cb-4ce8-85ce-cf68cf30343e'.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 3 documents.


Memories:
No relevant memories found.
Semantic Memories:
- You may like this blue dress. It is 
of good quality and great price.
Would you like to buy a 
waistband for your dress?
Great! I like this bule one. 
I will buy it for the party.
Medicine
David, a 38-year-old male with a history of allergies and sinus 
infections, has a family history of diabetes and hypertension. As 
a smoker of about one pack a day and an occasional drinker, 
his lifestyle choices may contribute to his health risks. After 
traveling to a tropical country where mosquito-borne illnesses 
are prevalent, he has experienced symptoms such as mild 
fatigue, headache, and muscle aches for the past week. 
For three days, I’ve had a 
fever ranging from 
100.5°F to 102°F, a rash 
on my limbs, joint pain 
and swelling, especially 
in my hands, episodes of 
diarrhea, abdominal pain, 
and nausea, leading to a 
loss of appetite.
The patient‘s travel history and 
symptoms suggest dengue fever, 
a mosquito-transmitted illnes

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:mem0.memory.main:Total existing memories: 0
INFO:mem0.vector_stores.mongodb:Retrieved document with ID '2a50db03-70cb-4ce8-85ce-cf68cf30343e'.


AI: Hello! How can I assist you today?


...

AI: That's great! Green tea is known for its numerous health benefits, including being high in antioxidants and potentially boosting metabolism. Do you have a favorite type of green tea or a particular brand that you enjoy?


INFO:mem0.vector_stores.mongodb:Retrieved document with ID '2a50db03-70cb-4ce8-85ce-cf68cf30343e'.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 2 documents.
INFO:mem0.vector_stores.mongodb:Retrieved document with ID '2a50db03-70cb-4ce8-85ce-cf68cf30343e'.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 3 documents.


Memories:
- Particularly likes green tea
- Likes to drink tea
Semantic Memories:
- In a narrow sense, the memory of the agent is only relevant to the historical information within
the same trial. Formally, for a given task, the historical information of the trial before step t is
ξt = {a1, o1, a2, o2, ..., at−1, ot−1}, and then the memory is derived based on ξt. In the above toy
example, for task (A), the agent at [step 3] needs to arrange the visiting order for Alice; at this time,
its memory contains the information about the selected attractions and arrival time in [step 1] and
[step 2]. For task (B), the agent has to choose a movie for Alice at [step 3]; at this time, its memory
contains the arranged time to watch films.
3.3 Broad Definition of the Agent Memory
In a broad sense, the memory of the agent can come from much wider sources, for example,
the information across different trials and the external knowledge beyond the agent-environment
interactions. Formally, given a series 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:mem0.memory.main:Total existing memories: 0
INFO:mem0.vector_stores.mongodb:Retrieved document with ID '2a50db03-70cb-4ce8-85ce-cf68cf30343e'.


AI: The definition of Agent Memory from the paper can be summarized as follows:

1. **Narrow Definition**: The memory of the agent is relevant to historical information within the same trial. For a given task, the memory is derived from the sequence of actions and observations before the current step.

2. **Broad Definition**: The memory can come from a wider range of sources, including information across different trials and external knowledge beyond the agent-environment interactions. This broader perspective emphasizes the importance of memory for the agent's evolving principles and applications.

These definitions highlight the role of memory in the agent's decision-making process and its capacity to learn from past experiences.
Goodbye!


In [14]:
coffee_search = memory.search(query="coffee brewing methods and preferences", user_id=CURRENT_USER_ID, limit=5)


INFO:mem0.vector_stores.mongodb:Retrieved document with ID '2a50db03-70cb-4ce8-85ce-cf68cf30343e'.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 2 documents.


In [16]:
coffee_search

{'results': [{'created_at': None,
   'agent_id': None,
   'updated_at': None,
   'id': '9d520037-eff0-48fe-9cb3-ecc949765cd0',
   'user_id': 'default_user',
   'actor_id': None,
   'run_id': None,
   'hash': None,
   'role': None,
   'data': 'Likes to drink tea',
   'score': 0.7095835208892822,
   'memory': 'Likes to drink tea',
   'metadata': {}},
  {'created_at': None,
   'agent_id': None,
   'updated_at': None,
   'id': '21d8fc88-a9bc-423d-8632-c0cd6c893c14',
   'user_id': 'default_user',
   'actor_id': None,
   'run_id': None,
   'hash': None,
   'role': None,
   'data': 'Particularly likes green tea',
   'score': 0.681547999382019,
   'memory': 'Particularly likes green tea',
   'metadata': {}}]}

In [15]:
paper_search = semantic_memory.search(query="What is the main idea of the paper?", user_id=CURRENT_USER_ID, limit=5)

INFO:mem0.vector_stores.mongodb:Retrieved document with ID '2a50db03-70cb-4ce8-85ce-cf68cf30343e'.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 5 documents.


In [17]:
paper_search

{'results': [{'created_at': None,
   'agent_id': None,
   'updated_at': None,
   'id': '57fd4245-ddb0-4f05-be2b-eb4b08ea08ce',
   'user_id': 'user-123',
   'actor_id': None,
   'run_id': None,
   'hash': None,
   'role': 'user',
   'data': 'LLMs\n[70–73]\nChallenges\nSecurity\nYao et al. [64], Shayegani et al.\n[65], Neel and Chang [66], Smith et al.\n[67], Dong et al. [68], Das et al. [69].\nExplainability Zhao et al. [63].\nBias & Fairness Gallegos et al. [60], Kotek\net al. [61], Li et al. [62].\nHallucination\nZhang et al. [53], Huang et al.\n[54], Rawte et al. [55], Ye\net al. [56], Ji et al. [57], Ton-\nmoy et al. [58], Jiang et al. [59].\nApplications\nRecommendation Li et al. [50], Lin et al.\n[51], Wang et al. [52].\nPsychology He et al. [49].\nFinance Li et al. [48].\nMedecine He et al. [45], Zhou et al.\n[46], Wang et al. [47].\nAutonomous Driving Cui et al. [43], Yang et al. [44].\nRobotics Zeng et al. [42].\nSoftware Engineering Fan et al. [39], Wang et al.\n[40], Zheng et